# SQL DATA TYPE

###  Содержание <a class="anchor" id=0></a>
- [1. Введение](#1)
- [2. Типы данных в PostgreSQL](#2)
- [3. Даты: основные типы](#3)
- [4. Функции и операторы для работы с датами](#4)
- [5. Строковые данные: основные типы](#5)
- [6. Функции и операторы для работы со строками](#6)
- [7. Итоги](#7)


## Введение <a class="anchor" id=1></a>

[к содержанию](#0)

В этом модуле мы будем работать с уже знакомой вам схемой `shipping`, хранящей таблицы с данными о компании, которая организует перевозки грузов.

Интересующие нас данные всё так же хранятся в таблицах `city`, `customer`, `driver` и `shipment` (таблица `truck` в этом модуле нам не понадобится).

<img src=sql_5_img1.jpg>



## Типы данных в PostgreSQL <a class="anchor" id=2></a>

[к содержанию](#0)

Особенностью хранения данных в БД является их строгая **типизация**, то есть точное и явное определение типов. Необходимость в типизации обусловлена тем, что компьютер по-разному обрабатывает даты, целые или дробные числа, строки.

Поэтому при создании таблицы БД обязательно указывают типы данных, которые будут сохраняться в каждом из её полей, — от этого зависит, какие значения допустимы в этих полях.

<img src=sql_5_img2.png>

Если поле таблицы определено **числовым типом данных**, то произвольную строку 'Всем привет!' в него записать уже **не получится**. Таким образом, типы данных — это один из основных видов ограничений в `PostgreSQL`.

[Официальная документация PostgreSQL](https://postgrespro.ru/docs/postgresql/9.5/ddl-constraints)

### ОСНОВНЫЕ ТИПЫ ДАННЫХ В POSTGRESQL

`SQL` — это язык со строгой типизацией, в котором каждый элемент данных имеет некоторый тип, определяющий его поведение и допустимое использование.

Типы данных в `PostgreSQL` можно разделить на несколько групп:

* `числовые типы` — для хранения чисел (целых и дробных);
* `типы даты/времени` — для хранения даты, времени, часовых поясов;
* `символьные типы` — для хранения символов или строк;
* `логический тип` — для хранения значений типа «истина», «ложь».

>Каждая группа (кроме логического типа) объединяет несколько типов, отличающихся по допустимому диапазону хранимых данных. Например, числовые типы данных хранят целые числа, дробные числа, а строковые — подразделяются на типы с фиксированной и переменной длиной. А скажем, целочисленный тип данных integer может хранить значения в диапазоне от -2147483648 до 2147483647.

Конечно, чем больше допустимый диапазон, тем больше требуется места на жёстком диске для хранения каждого поля выбранного типа.

Кроме возможных значений из доступного диапазона, поле независимо от выбранного типа может принимать значение `NULL` (отсутствующее значение).

Напоминаем, что `NULL` отличается от нулевого значения или поля, содержащего пробел или пустую строку.

### И ЕЩЁ РАЗ О ВАЖНОСТИ ТИПИЗАЦИИ

В зависимости от требований к хранимой информации необходимо правильно применять типы данных. На это есть как минимум две причины.

1. Разные типы данных могут занимать разный объём памяти.

Например, если заранее известно, что поле таблицы будет принимать только небольшие числовые значения от 1 до 10, то можно не задавать для него тип bigint с максимально возможными хранимыми значениями, достаточно будет типа smallint (эти типы не будут предметом нашего обсуждения, но вы может посмотреть их описание в таблице).

2. На преобразование типов данных тратится время.

В некоторых случаях может потребоваться преобразование одного типа в другой — например, если нужно извлечь число из строки и произвести с ним какие-либо действия. Для подобных преобразований нужно использовать дополнительные функции. Иногда без этого не обойтись, но по возможности лучше хранить данные в полях соответствующего типа.

В этом модуле мы ограничимся разговором о таких типах данных, как даты и строки, а также поговорим о функциях, которые позволяют с ними работать.

[Документация по типам данных](https://postgrespro.ru/docs/postgrespro/9.5/datatype)

## Даты: основные типы <a class="anchor" id=3></a>

[к содержанию](#0)

Для них в `Postgres` существует несколько типов данных — все они представлены в таблице ниже.

<img src=sql_5_img3.png>

### TIMESTAMP

`Timestamp` — наиболее распространённый тип данных, так как он содержит и дату, и время, а также используется в любых логах событий, временных рядах и в большинстве системных таблиц.

Согласно [стандарту ISO](https://ru.wikipedia.org/wiki/ISO_8601), значение выглядит как "`2019-07-14 01:35:44.702165+00`", где перечислены через точку год-месяц-день, время и часовой пояс.

Для получения текущего значения даты и времени в `Postgres` используются функции `CURRENT_TIMESTAMP` (есть в стандарте `SQL`) и `NOW()` (есть в большинстве баз данных).

In [ ]:
SELECT NOW()
/* равносильно */
SELECT CURRENT_TIMESTAMP

### TIMESTAMP WITH TIME ZONE

>`Timestamp with time zone` позволяет хранить сведения о часовом поясе, что может быть удобно при анализе географически распределённых временных данных для единообразия хранения.

Предположим, в вашей компании в базу подтягивается время прихода сотрудников на работу. Вы пришли в 10 утра по Москве, а в Екатеринбурге в это время — полдень. Чтобы ваши коллеги из Екатеринбурга поняли, что вы пришли на работу вовремя, им нужно помнить про разницу в часовых поясах. А теперь представим, что пользователи БД разбросаны по всему миру и всем им необходимо помнить о разнице во времени и учитывать её при сверках с другими регионами.

Согласитесь, неудобно. Поэтому временные метки лучше всего хранить в базе в едином часовом поясе.

Посмотрим, как работает этот тип данных.

Сначала попробуем узнать, в каком часовом поясе выводятся временные данные в настоящий момент. Для этого выполните команду

In [ ]:
show timezone

В результате вы увидите GMT (среднее время по Гринвичу) — это наиболее частая установка по умолчанию для баз данных.
Список часовых поясов можно увидеть в системном справочнике `pg_timezone_names`.

<img src=sql_5_img4.png>

Посмотрите содержимое этого справочника в `Metabase`.

А ещё посмотрите ваше время в каком-нибудь часовом поясе, например, в Москве. Для этого выполните в `Metabase` запрос

In [ ]:
select now() at time zone 'Europe/Moscow'

Указание `at time zone` позволяет переводить дату/время без часового пояса в дату/время с часовым поясом и обратно, а также пересчитывать значения времени для различных часовых поясов.

В таблице ниже приведены примеры того, как работает `at time zone` для разных типов данных.

<img src=sql_5_img5.png>


### DATE

С типом `date` вы уже знакомы, его реализация предельно проста. Отметим только, что тип `timestamp` (`with/without time zone`) можно легко перевести в соответствующую дату, используя синтаксис

In [ ]:
/* тип timestamp переводится в дату */
"timestamp_column"::date
/* и наооборот */
"date_column"::timestamp
/* для получения текущей даты */
select CURRENT_DATE
/* или */ 
select now()::date

Предположим, у нас есть дата и время какого-то события и мы хотим посмотреть, к какой дате оно относится для Москвы и для UTC.

Столбцы в выдаче: `dt_msk` (дата в московском часовом поясе), `dt_utc` (дата в UTC).

In [ ]:
WITH x AS 
(
SELECT '2018-12-31 21:00:00+00'::timestamp WITH time zone ts
)
SELECT 
        (ts at time zone 'Europe/Moscow')::date dt_msk,
        (ts)::date dt_utc
FROM x

### INTERVAL

`Interval` — тип данных, позволяющий хранить разницу между двумя временными метками. 

Интервалы хранят данные в трёх отдельных полях — месяцах, днях, секундах. Это сделано из-за того, что количество дней в месяце и часов в дне может быть разным. Пример значения такого типа: "`195 days -10:52:23.563955`".

Подробнее о том, как работают интервалы, мы узнаем позже, на примере конкретных функций, с помощью которых можно вычислять эти интервалы и работать с ними.

## Функции и операторы для работы с датами <a class="anchor" id=4></a>

[к содержанию](#0)

## Строковые данные: основные типы <a class="anchor" id=5></a>

[к содержанию](#0)

## Функции и операторы для работы со строками <a class="anchor" id=6></a>

[к содержанию](#0)

## Итоги <a class="anchor" id=7></a>

[к содержанию](#0)